In [1]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import coralme
from coralme.builder.main import MEBuilder, MEReconstruction, METroubleshooter
from coralme.builder.curation import MECurator, MEManualCuration

In [3]:
org = 'bsubtilis'
organism = './{:s}/organism.json'.format(org)
inputs = './{:s}/input.json'.format(org)

### Full

In [4]:
builder = MEBuilder(*[organism, inputs])
builder.configuration['run_bbh_blast'] = False

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16


In [5]:
builder.generate_files(overwrite=True)

Initiating file processing...
~ Processing files for bsubtilis...


Checking M-model metabolites...                                            : 100.0%|██████████|   990/  990 [00:00<00:00]
Checking M-model genes...                                                  : 100.0%|██████████|   844/  844 [00:00<00:00]
Checking M-model reactions...                                              : 100.0%|██████████|  1250/ 1250 [00:00<00:00]
Generating complexes dataframe from optional proteins file...              : 100.0%|██████████|  4554/ 4554 [00:00<00:00]
Syncing optional genes file...                                             : 100.0%|██████████|  4541/ 4541 [00:00<00:00]
Looking for duplicates within datasets...                                  : 100.0%|██████████|     5/    5 [00:00<00:00]
Gathering ID occurrences across datasets...                                : 100.0%|██████████| 10647/10647 [00:00<00:00]
Solving duplicates across datasets...                                      : 0.0%|          |     0/    0 [00:00<?]
Pruning GenBank...            

unknown metabolite '4crsol_c' created
unknown metabolite 'dad__5_c' created
unknown metabolite 'dhgly_c' created
unknown metabolite '5drib_c' created
unknown metabolite 'cu_e' created
unknown metabolite 'cu_c' created
unknown metabolite 'cbl1_e' created
unknown metabolite 'cbl1_c' created


Adding manual curation of complexes...                                     : 100.0%|██████████|   438/  438 [00:00<00:00]
Getting sigma factors...                                                   : 100.0%|██████████|    38/   38 [00:00<00:00]
Getting generics from Genbank contigs...                                   : 100.0%|██████████|     6/    6 [00:00<00:00]
Getting TU-gene associations from optional TUs file...                     : 100.0%|██████████|  1647/ 1647 [00:00<00:00]
Adding protein location...                                                 : 100.0%|██████████|  4683/ 4683 [00:00<00:00]
Purging M-model genes...                                                   : 100.0%|██████████|   851/  851 [00:00<00:00]
Getting enzyme-reaction associations...                                    : 100.0%|██████████|  1262/ 1262 [00:00<00:00]


Reading bsubtilis done.


...

~ Processing files for iJL1678b...


Checking M-model metabolites...                                            : 100.0%|██████████|  1660/ 1660 [00:00<00:00]
Checking M-model genes...                                                  : 100.0%|██████████|  1271/ 1271 [00:00<00:00]
Checking M-model reactions...                                              : 100.0%|██████████|  2377/ 2377 [00:00<00:00]
Looking for duplicates within datasets...                                  : 100.0%|██████████|     5/    5 [00:00<00:00]
Gathering ID occurrences across datasets...                                : 100.0%|██████████|  8517/ 8517 [00:00<00:00]
Solving duplicates across datasets...                                      : 0.0%|          |     0/    0 [00:00<?]
Getting sigma factors...                                                   : 100.0%|██████████|     7/    7 [00:00<00:00]
Getting TU-gene associations from optional TUs file...                     : 100.0%|██████████|  1647/ 1647 [00:00<00:00]
Adding protein location...    

Reading iJL1678b done.


Updating translocation machinery from homology...                          : 100.0%|██████████|     9/    9 [00:00<00:00]
Updating protein location from homology...                                 : 100.0%|██████████|   514/  514 [00:00<00:00]
Updating translocation multipliers from homology...                        : 100.0%|██████████|     3/    3 [00:00<00:00]
Updating lipoprotein precursors from homology...                           : 100.0%|██████████|    14/   14 [00:00<00:00]
Updating cleaved-methionine proteins from homology...                      : 100.0%|██████████|   343/  343 [00:00<00:00]
Mapping M-metabolites to E-metabolites...                                  : 100.0%|██████████|   147/  147 [00:00<00:00]
Updating generics from homology...                                         : 100.0%|██████████|    10/   10 [00:00<00:00]
Updating folding from homology...                                          : 100.0%|██████████|     2/    2 [00:00<00:00]
Updating ribosome subrea

File processing done.


In [6]:
builder.build_me_model(overwrite=False)

Initiating ME-model reconstruction...


Adding biomass constraint(s) into the ME-model...                          : 100.0%|██████████|    11/   11 [00:00<00:00]

Read LP format model from file /tmp/tmpqmh5x9g_.lp
Reading time = 0.00 seconds
: 998 rows, 2524 columns, 10562 nonzeros



Adding Metabolites from M-model into the ME-model...                       : 100.0%|██████████|  1088/ 1088 [00:00<00:00]
Adding Reactions from M-model into the ME-model...                         : 100.0%|██████████|  1277/ 1277 [00:00<00:00]
Adding Transcriptional Units into the ME-model...                          : 100.0%|██████████|  1515/ 1515 [00:09<00:00]
Adding features from contig NC_000964.3 into the ME-model...               : 100.0%|██████████|  4537/ 4537 [00:08<00:00]
Adding features from contig BSU_misc_RNA_38 into the ME-model...           : 100.0%|██████████|     2/    2 [00:00<00:00]
Adding features from contig G8J2-183 into the ME-model...                  : 100.0%|██████████|     2/    2 [00:00<00:00]
Adding features from contig G8J2-180 into the ME-model...                  : 100.0%|██████████|     2/    2 [00:00<00:00]
Adding features from contig G8J2-181 into the ME-model...                  : 100.0%|██████████|     2/    2 [00:00<00:00]
Adding features from co

ME-model was saved in the ./bsubtilis/ directory as MEModel-step1-bsubtilis.pkl


Adding tRNA synthetase(s) information into the ME-model...                 : 100.0%|██████████|   306/  306 [00:00<00:00]
Adding tRNA modification SubReactions...                                   : 100.0%|██████████|    19/   19 [00:00<00:00]
Associating tRNA modification enzyme(s) to tRNA(s)...                      : 100.0%|██████████|    33/   33 [00:00<00:00]
Adding SubReactions into TranslationReactions...                           : 100.0%|██████████|  4328/ 4328 [00:01<00:00]
Adding RNA Polymerase(s) into the ME-model...                              : 100.0%|██████████|    39/   39 [00:00<00:00]
Associating a RNA Polymerase to each Transcriptional Unit...               : 100.0%|██████████|  1515/ 1515 [00:00<00:00]
Processing ComplexData in ME-model...                                      : 100.0%|██████████|   322/  322 [00:00<00:00]
Adding ComplexFormation into the ME-model...                               : 100.0%|██████████|  5054/ 5054 [00:00<00:00]
Adding SubReactions into

ME-model was saved in the ./bsubtilis/ directory as MEModel-step2-bsubtilis.pkl
ME-model reconstruction is done.
Number of metabolites in the ME-model is 4633 (+364.23%, from 998)
Number of reactions in the ME-model is 7760 (+514.90%, from 1262)
Number of genes in the ME-model is 1154 (+35.61%, from 851)


In [7]:
builder.troubleshoot(growth_key_and_value = { builder.me_model.mu : 0.001 })

The MINOS and quad MINOS solvers are a courtesy of Prof Michael A. Saunders. Please cite Ma, D., Yang, L., Fleming, R. et al. Reliable and efficient solution of genome-scale models of Metabolism and macromolecular Expression. Sci Rep 7, 40863 (2017). https://doi.org/10.1038/srep40863

~ Troubleshooting started...
  Checking if the ME-model can simulate growth without gapfilling reactions...
  Original ME-model is feasible with a tested growth rate of 0.001000 1/h
~ Final step. Fully optimizing with precision 1e-6 and save solution into the ME-model...
  Gapfilled ME-model is feasible with growth rate 0.176847 (M-model: 0.117966).
ME-model was saved in the ./bsubtilis/ directory as MEModel-step3-bsubtilis-TS.pkl


### Quick

In [ ]:
builder = MEBuilder(*['./{:s}/coralme-config.yaml'.format(org)])

In [ ]:
builder.build_me_model(overwrite=False, update=True, prune=False)

In [ ]:
builder.troubleshoot(growth_key_and_value = { builder.me_model.mu : 0.001 })

### Load builder

In [ ]:
builder = MEBuilder(*['./{:s}/coralme-config.yaml'.format(org)])
builder.me_model = builder.load(builder.configuration['out_directory']+'MEModel-step3-{}-TS.pkl'.format(org))

### Find issue

In [ ]:
builder.find_issue('')

### Curation notes

In [ ]:
builder.curation_notes